# GTEx eQTL detection: trans-eQTLs for v8 consortium paper

This notebook summarizes the trans- associations that we draw from the GTEx data, for the consortium paper.

## All the relevant code for this pipeline can be viewed at:

### https://github.com/bee-hive/RNAseq_pipeline/tree/master/Scripts/eqtls/trans/gtex/v8_consortium/

## 1. Pre-processing steps

For the GTEx v8 trans-eQTL mapping pipeline, there are a few pre-processing steps. They involve:
- Getting the genotype files ready (refer to the Genotype v8 processing notebook for this: https://github.com/bee-hive/RNAseq_pipeline/blob/master/Analysis/Notebooks/genotype/GTEx_v8_genotype_processing.ipynb
- Getting the expression files ready
- Getting the gene mappability and cross-mapping filters ready

The first section will work out the gene mappability and cross-mappability filters for the trans-eQTL pipeline.

### 1.1 Getting the gene mappability and cross-mapping filters ready

As of this writing, there is no ENCODE mappability track for hg38. Fortunately, we can use another function from the [GEM library](http://gemlibrary.sourceforge.net/), <code>gem-mappability</code>

Note that this pipeline was made referring to the following post: https://www.biostars.org/p/181014/

I have downded the latest GEM library into the directory:

<code>
/tigress/BEE/bin/GEM-binaries-Linux-x86_64-core_i3-20130406-045632/
</code>

In order to use the GEM library, we first need to index the hg38 fasta file, and then run the gem-mappability function with the indexed genome.

<code>
https://github.com/bee-hive/RNAseq_pipeline/tree/master/Scripts/processing/mappability/slurm/gem_mappability.slurm
</code>

In [ ]:
%%bash

# Index the hg38 genome, and calculate the mappability for 75mers and 36mers

sbatch /tigress/BEE/RNAseq/Scripts/processing/mappability/gem-mappability.slurm

This step creates the <code>hg38_75mer</code> and <code>hg38_36mer</code> mappability files. Now, we need to use the GEM functionality, <code>gem-2-wig</code> to convert this to wig format, then use <code>wigToBigWig</code> to convert to bigWig format, and finally use the <code>bigWigToBedGraph</code> to convert this file to the bed format, for the mappability and cross-mapping scripts. Unfortunately, we need to divide this step into 2, since the wig output has suffix 'AC' to the chromosome names - I found that we need to delete them separately for the wigTobigWig format to work properly.

<code>
https://github.com/bee-hive/RNAseq_pipeline/tree/master/Scripts/processing/mappability/slurm/gem_to_wig.slurm
</code>
<code>
https://github.com/bee-hive/RNAseq_pipeline/tree/master/Scripts/processing/mappability/slurm/mod_lines.py
</code>

Also note that for the <code>bigWigToBedGraph</code> function, I followed the conda install instruction: https://bioconda.github.io/recipes/ucsc-bigwigtobedgraph/README.html

<code>
https://github.com/bee-hive/RNAseq_pipeline/tree/master/Scripts/processing/mappability/slurm/bigWig_to_BedGraph.slurm
</code>

In [ ]:
%%bash

# Conver the mappability file to wig, then to bigWig, then to bed

sbatch /tigress/BEE/RNAseq/Scripts/processing/mappability/gem_to_wig.slurm
nohup python /tigress/BEE/RNAseq/Scripts/processing/mappability/mod_lines.py &
sbatch /tigress/BEE/RNAseq/Scripts/processing/mappability/gem_to_wig.slurm

Now that we have the bed files, we can follow the same mappability file generation pipeline for GTEx v6p paper, coming from the Battle group. The scripts for this pipeline is proprietary, so I'll just include the outline here without sharing the codebase:

- Create separate files for exon and utr annotations.
- Compute average mappability of genes.
- Save multimapped k-mers (75-mer from exon or 36-mer from utr that align to multiple positions in the genome i.e. alignability < 1) of each gene in a separate file in a given directory.
- Compute cross-mappabile gene pairs (excuse the file name).

Following these steps lead to the generation of gene mappability and cross-mapping gene pair lists, which can be used as filters in the trans-eQTL pipeline.

In [ ]:
# Needed to download the fasta files for each chromosome 
import subprocess
import os

os.chdir('/tigress/BEE/RNAseq/Data/Resources/reference/hg38/')

chrs = [str(i+1) for i in range(22)]
chrs = chrs + ['X','Y','M']

for i in chrs:
    subprocess.run(['wget', 'http://hgdownload.cse.ucsc.edu/goldenPath/hg38/chromosomes/chr' + i + '.fa.gz'])

# Also needed to download the bowtie index for hg38
os.chdir('/tigress/BEE/bin/bowtie_index/')
subprocess.run(['wget', 'ftp://ftp.ccb.jhu.edu/pub/data/bowtie_indexes/GRCh38_no_alt.zip'])


After this step, there are two annotation files saved for the trans-eQTL pipeline:

<code>
/tigress/BEE/RNAseq/Output/processing/mappability/annotation/hg38_gene_mappability.txt
</code>

which has the average gene mappability for each gene in hg38, and

<code>
/tigress/BEE/RNAseq/Output/processing/mappability/annotation/hg38_mappability_conflicts.txt
</code>

which has the cross-mapping conflict gene pairs for hg38. Using these annotations, we can go ahead with the trans-eQTL pipeline. Note that I created a graph version of this cross-mapping conflict file, which is the file that I actually use the for trans pipeline:

<code>
/tigress/BEE/RNAseq/Output/processing/mappability/annotation/hg38_mappability_graph.txt
</code>

## 2. trans-eQTL mapping pipeline

The simplest way of looking for trans-eQTLs is to simply plug in all gene expression values and all SNPs and looking for associations. However, we need to be more careful for trans- tests to make sure that we are properly filtering out false positives, since the weaker signal and higher number of tests can lead to mapping artifacts overestimating the strength of association.

We are going to have the following filters:

1. Filter genes based on mappability - we use a threshold of 0.8.
2. Filter SNPs based on RepeatMasker (not belonging to any annotated repeat regions) - this part is covered in the Genotype notebook.
3. Cross out any SNP-gene pairs for which there is potential cross-mapping


- Repeatmasker raw data, which annotates all repeat regions in human genome version hg19 (including SINEs, LINEs, transposons, etc.), can be found in :

<code>/tigress/BEE/RNAseq/Data/Resources/RepeatMasker/</code>

- MatrixEQTL runs, in addition to the tissue-specific PEER factors, also include a set covariates: First 5 genotype PCs, sex, genotyping platform, and pcr platform. Either sex or platform is not included if there is only one type (i.e. testis, cervix, etc.)

As it can be seen, we put a lot of filters in place to make sure that we get a conservative set of trans-eQTL calls.

### 2.1 MatrixEQTL scripts for all-by-all trans tests

In [ ]:
%%bash
export proj='/tigress/BEE/RNAseq'
# gtex gct
python $proj/Scripts/eqtls/trans/gtex/v8_consortium/v8_trans_matrix_eqtl_wrapper.py \
/Data/Expression/gtex/hg38/GTEx_Analysis_v8_eQTL_expression_matrices/ \
.v8.normalized_expression.bed.gz /Output/trans-mapping/gtex/MatrixEQTL/v8/all-by-all/ \
/Output/joblogs/trans-mapping/gtex/MatrixEQTL/v8/ v8_trans_matrix_eqtl.R 2.5e8 25000

Also refer to the following scripts:

<code>https://github.com/bee-hive/RNAseq_pipeline/blob/master/Scripts/eqtls/trans/gtex/v8_consortium/v8_trans_matrix_eqtl_wrapper.py</code>

<code>https://github.com/bee-hive/RNAseq_pipeline/blob/master/Scripts/eqtls/trans/gtex/v8_consortium/v8_trans_matrix_eqtl.R</code>

After running the above jobs, we have raw output files in Della, located at:

<code>
/tigress/BEE/RNAseq/Output/trans-mapping/gtex/MatrixEQTL/v8/all-by-all/
</code>

#### Outputs : Number of genes tested, SNPs tested, Total Number of tests, P-value histograms, eQTL summary statistics below p-value threshold of 1e-5 - 236M for 6 tissues

The tissue set that I tested with is:

<code>
Adipose_Subcutaneous, Cells_EBV-transformed_lymphocytes, Skin_Sun_Exposed_Lower_leg, Testis, Thyroid, Whole_Blood
</code>

### 2.2 FDR calibration for trans-eQTL mapping

After running the MatrixEQTL for all SNP-gene pairs, we collect the overall statistics across parallel runs, and calibrate the FDR in order to assess power. For all-by-all testing, we run Benjamini-Hochberg procedure to calculate FDR. (The null distribution actually shows a depletion of significant hits, which can be shown through p-value histograms and qq-plots, probably due to rigorous correction of covariates)


In [ ]:
%%bash
# gtex gct
Rscript $proj/Scripts/eqtls/trans/gtex/v8_consortium/v8_MatrixEQTL_FDR_control.R


Also refer to the following script:

<code>https://github.com/bee-hive/RNAseq_pipeline/blob/master/Scripts/eqtls/trans/gtex/v8_consortium/v8_MatrixEQTL_FDR_control.R</code>

Location of resulting eQTL and eGene lists in Della :

<code>
/tigress/BEE/RNAseq/Output/trans-mapping/gtex/trans-eQTLs/MatrixEQTL/all-by-all-PEER-increments/eqtl_list/
</code>

#### Outputs : SNP-gene pair trans-eQTLs at FDR 0.05 and 0.10, eGene list at FDR 0.05 and 0.10

The following output shows six numbers:
trans-eQTL pairs at FDR 5%, 10%
unique genes at FDR 5%, 10%
unique SNPs at FDR 5%, 10%

<code>
"Adipose_Subcutaneous"
[1] 398
[1] 524
[1] 94
[1] 131
[1] 366
[1] 473
[1] "Cells_EBV-transformed_lymphocytes"
[1] 55
[1] 89
[1] 15
[1] 24
[1] 55
[1] 88
[1] "Skin_Sun_Exposed_Lower_leg"
[1] 967
[1] 1213
[1] 172
[1] 233
[1] 862
[1] 1084
[1] "Testis"
[1] 1918
[1] 2447
[1] 252
[1] 380
[1] 1463
[1] 1865
[1] "Thyroid"
[1] 1092
[1] 1296
[1] 167
[1] 234
[1] 720
[1] 853
[1] "Whole_Blood"
[1] 566
[1] 714
[1] 126
[1] 169
[1] 512
[1] 646
</code>

#### With the increased sample size, we have good power to detect trans-eQTLs in GTEx tissues.

## 3. Re-run the trans-eQTL pipeline with the updated cross-mapping file and genotype file (SHAPEIT2 imputed genotypes)

In [ ]:
%%bash
export proj='/tigress/BEE/RNAseq'
# gtex gct
python $proj/Scripts/eqtls/trans/gtex/v8_consortium/v8_trans_matrix_eqtl_wrapper.py \
/Data/Expression/gtex/hg38/GTEx_Analysis_v8_eQTL_expression_matrices/ \
.v8.normalized_expression.bed.gz /Output/trans-mapping/gtex/MatrixEQTL/v9/all-by-all/ \
/Output/joblogs/trans-mapping/gtex/MatrixEQTL/v8/ v9_trans_matrix_eqtl.R 2.5e8 50000